In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import numpy as np
from parse_reports import *
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
import pandas as pd


In [ ]:
module_name_dict = {
    "Activation Memory" : "activation",
    "Feature Loader"    : "feature",
    "Output Aligner"    : "mm",
    "Output Scaler"  : "output",
    "Padding Module" : "padder",
    "Write Queue"   : "piso",
    "Controller"    : "qracc",
    "SeqAcc (Digital)" : "seq",
    "WSAcc": "wsacc",
}

module_name_dict_full = {
    'u_activation_buffer': 'Activation Memory',
    'u_csr': 'Controller',
    'u_feature_loader': 'Feature Loader',
    'u_mm_output_aligner': 'Output Aligner',
    'u_output_scaler_set': 'Output Scaler',
    'u_padder': 'Padding Module',
    'u_piso_write_queue': 'Write Queue',
    'u_qracc_controller': 'Controller',
    'u_seq_acc': 'SeqAcc (Digital)',
    'u_wsacc': 'WSAcc',
}

In [ ]:

# --- Power Report ---
power_report_path = "/home/lquizon/lawrence-workspace/SRAM_test/qrAcc2/qr_acc_2_digital/results/base_with_sram_1/reports/power_qr_acc_top.txt"
df_power = parse_report(power_report_path)
# Clean up the hierarchy names
df_power['hierarchy_clean'] = df_power['hierarchy'].replace(module_name_dict_full)
df_power_level1 = df_power[df_power['hierarchy_level'] == 1]

# --- Area Report ---
area_report_path = "/home/lquizon/lawrence-workspace/SRAM_test/qrAcc2/qr_acc_2_digital/results/base_with_sram_1/reports/area_qr_acc_top.txt"
df_area = parse_report(area_report_path)
df_area['hierarchy_clean'] = df_area['hierarchy'].replace(module_name_dict_full)
df_area_level1 = df_area[df_area['hierarchy_level'] == 1]

In [ ]:
plot_power_pie(df_power_level1)
plt.savefig('power_pie.svg', bbox_inches='tight')

In [ ]:
plot_power_report(df_power_level1)
plt.savefig('power_bar.svg', bbox_inches='tight')

In [ ]:
dfa = parse_area_report("/home/lquizon/lawrence-workspace/SRAM_test/qrAcc2/qr_acc_2_digital/results/base_with_sram_1/reports/area_qr_acc_top.txt")
dfa_level1 = dfa[dfa['hierarchy_level'] == 1]
dfa_level1

In [ ]:
fig, ax = plt.subplots(figsize=(4, 2), dpi=300)
plot_area_report(df_area_level1, ax=ax)
plt.savefig('area_bar.svg', bbox_inches='tight')

In [ ]:
plot_area_pie(df_area_level1,-30)
plt.savefig('area_pie.svg', bbox_inches='tight')

# Per-state Power

In [ ]:
power_path_dicts = {
    'Compute Analog': 'per_state_primetime/power/computeanalog_power_pt.rpt',
    'Compute Digital': 'per_state_primetime/power/computedigital_power_pt.rpt',
    'Idle': 'per_state_primetime/power/idle_power_pt.rpt',
    'Load Acts': 'per_state_primetime/power/loadacts_power_pt.rpt',
    'Load Bias': 'per_state_primetime/power/loadbias_power_pt.rpt',
    'Load Scalers': 'per_state_primetime/power/loadscaler_power_pt.rpt',
    'Load Weights': 'per_state_primetime/power/loadweights_power_pt.rpt',
    'Load Weights (WSAcc)': 'per_state_primetime/power/loadweightsdigital_power_pt.rpt',
    'Read Acts': 'per_state_primetime/power/readacts_power_pt.rpt',
}

# Parse the power reports for each state into a single DataFrame
df_power_states = pd.DataFrame()
for state, path in power_path_dicts.items():
    state_df = parse_report(path)
    state_df['state'] = state
    df_power_states = pd.concat([df_power_states, state_df], ignore_index=True)
df_power_states.to_csv('per_state_power.csv', index=False)

In [ ]:
# Get the clean CSV for the manuscript
# CSV contains only module name, state, and the 4 power columns
# Also includes only hierarchy level 1

# Clean the names of the columns
column_dict = {
    'Module': 'module_name',
    'State': 'state',
    'Internal Power': 'int_power',
    'Switching Power': 'switch_power',
    'Leakage Power': 'leak_power',
    'Total Power': 'total_power'
}

module_name_dict = {
    "Activation Memory" : "activation",
    "Feature Loader"    : "feature",
    "Output Aligner"    : "mm",
    "Output Scaler"  : "output",
    "Padding Module" : "padder",
    "Write Queue"   : "piso",
    "Controller"    : "qracc",
    "SeqAcc (Digital)" : "seq",
    "WSAcc": "wsacc",
}

# Reverse the entries of the dictionary
module_name_dict = {v: k for k, v in module_name_dict.items()}

df_power_states_clean = df_power_states[df_power_states['hierarchy_level'] == 1]
df_power_states_clean = df_power_states_clean[['module_name', 'state', 'total_power']]
# Use engineering format
# df_power_states_clean['total_power'] = df_power_states_clean['total_power'].apply(lambda x: np.around(x, 2)) 
# Use module name as index, and state as columns
# If there are duplicate (module_name, state) pairs, aggregate them (e.g., sum or mean)
df_power_states_clean = df_power_states_clean.groupby(['module_name', 'state'], as_index=False).agg({'total_power': 'first'})
df_power_states_clean = df_power_states_clean.pivot(index='module_name', columns='state', values='total_power')
# Rename index to 'Module'
df_power_states_clean.index.name = 'Module'
# Add total power row
# Rename the index to the module names
df_power_states_clean.index = df_power_states_clean.index.map(module_name_dict)
df_power_states_clean.loc['Total Power'] = df_power_states_clean.sum(numeric_only=True)

# Add a column for the average power of each module
df_power_states_clean.to_csv('per_state_power_clean.csv')
df_power_states_clean['Average Power'] = df_power_states_clean.mean(axis=1)

df_power_states_clean.to_csv('per_state_power_clean_withavg.csv')
df_power_states_clean

In [ ]:
# Stacked bar plot of the per-state power
df_power_states_clean = pd.read_csv('per_state_power_clean.csv', index_col=0)
df_power_states_clean = df_power_states_clean.drop('Total Power')
df_power_states_clean = df_power_states_clean.T
fig = plt.figure(figsize=(5, 3), dpi=300)
ax = fig.add_subplot(111)
df_power_states_clean.plot(kind='bar', stacked=True,
                          title='Per-State Power Consumption',
                          ylabel='Power (W)',
                          xlabel='State',
                          colormap='tab20',
                          width=0.8,
                          ax=ax)
# set legend outside the plot
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Module')
# Use engformatter for y-axis
from matplotlib.ticker import EngFormatter
eng_format = EngFormatter()
ax.yaxis.set_major_formatter(eng_format)
plt.xticks(rotation=30, ha='right')

In [ ]:
# Bar plot of the average power of each module
df_power_states_clean = pd.read_csv('per_state_power_clean_withavg.csv', index_col=0)
df_power_states_clean = df_power_states_clean.drop('Total Power')
df_power_states_clean = df_power_states_clean.sort_values(by='Average Power', ascending=False)
fig = plt.figure(figsize=(5, 3), dpi=300)
ax = fig.add_subplot(111)
# turn index into a column
df_power_states_clean = df_power_states_clean.reset_index()
df_power_states_clean['hierarchy_clean'] = df_power_states_clean['Module']
df_power_states_clean['total_power'] = df_power_states_clean['Average Power']
# Add a percentage column
df_power_states_clean['percentage'] = df_power_states_clean['Average Power'] / df_power_states_clean['Average Power'].sum() * 100

plot_power_report(df_power_states_clean, ax=ax)

In [ ]:
plot_power_pie(df_power_states_clean, 45)